### LSE Data Analytics Online Career Accelerator 
# Course 301: Data Analytics with Python

## Practical activity: Conduct A/B testing in Python

**This is the solution to the activity.**

An online bicycle store has changed their home page interface to encourage visitors to click through to their loyalty programme sign-up page. They are hoping the new interface will encourage more visitors to access the loyalty programme page, see what benefits the programme brings, and hopefully then sign up. The current click-through rate sits at around 50% annually, and the company hopes the new design will push this to at least 55%. 

This analysis uses the `new_bike_shop_AB.csv` data set. Using your Python and data wrangling skills, you will run an A/B test on the data to measure the significance of the interface change based on click-through rates to the loyalty programme page. 

## 1. Prepare your workstation

In [ ]:
# import the necessary libraries
import statsmodels.stats.api as sms
from statsmodels.stats.power import TTestIndPower

## 2. Perform power analysis

In [ ]:
# perform the power analysis to determine sample size
effect = sms.proportion_effectsize(0.50, 0.55)   
 
effect,
alpha = 0.05
power = 0.8

analysis = TTestIndPower()
result = analysis.solve_power(effect, power=power, nobs1=None, ratio=1.0, alpha=alpha)

print('Sample Size: %.3f' % result)

## 3. Import data set

In [ ]:
# import the necessary libraries
import pandas as pd
import math
import numpy as np
import statsmodels.stats.api as sms
import scipy.stats as st
import matplotlib as mpl
import matplotlib.pyplot as plt

In [ ]:
# read the data set with Pandas
df = pd.read_csv('new_bike_shop_AB.csv')

# print the DataFrame
print(df.shape)
df.head()

In [ ]:
# view DataFrame
df.info()

## 4. Clean the data

In [ ]:
# rename the columns
df_new = df.rename(columns={'IP Address': 'IPAddress', 'LoggedInFlag': "LoyaltyPage"})

# view the DataFrame
print(df_new.shape)
df_new.head()

In [ ]:
# drop duplicate values
df_new.drop_duplicates(subset ="IPAddress",
                       keep = False, inplace = True)


# drop duplicate columns
df_final = df_new.drop(["Unnamed: 0", "RecordID", "VisitPageFlag"],
                       axis=1)


# view the DataFrame
print(df_final.shape)

df_final.info()

## 5. Subset the DataFrame

In [ ]:
# split data set into ID1 as treatment and ID2 & ID3 as control group
df_final['Group'] = df_final['ServerID'].map({1:'Treatment', 2:'Control', 3:'Control'})

# view DataFrame
print(df_final.shape)
df_final.head()

In [ ]:
# count the values
df_final["Group"].value_counts()

In [ ]:
# Create two DataFrames
control_sample = df_final[df_final['Group'] == 'Control'].sample(n=1565,
                                                                 random_state=22) 

treatment_sample = df_final[df_final['Group'] == 'Treatment'].sample(n=1565,
                                                                       random_state=22)

# view the DataFrames
print(control_sample)
print(treatment_sample)

## 6. Perform A/B testing

In [ ]:
# perform A/B testing
# create variable and merge DataFrames
ab_test = pd.concat([control_sample, treatment_sample], axis=0)

ab_test.reset_index(drop=True, inplace=True)

# view output
ab_test

In [ ]:
# calculate the conversion rates
conversion_rates = ab_test.groupby('Group')['LoyaltyPage']


# Standard deviation of the proportion
STD_p = lambda x: np.std(x, ddof=0)    
# Standard error of the proportion
SE_p = lambda x: st.sem(x, ddof=0)     

conversion_rates = conversion_rates.agg([np.mean, STD_p, SE_p])
conversion_rates.columns = ['conversion_rate', 'std_deviation', 'std_error']


conversion_rates.style.format('{:.3f}')

In [ ]:
from statsmodels.stats.proportion import proportions_ztest, proportion_confint

control_results = ab_test[ab_test['Group'] == 'Control']['LoyaltyPage']
treatment_results = ab_test[ab_test['Group'] == 'Treatment']['LoyaltyPage']

n_con = control_results.count()
n_treat = treatment_results.count()

successes = [control_results.sum(), treatment_results.sum()]

nobs = [n_con, n_treat]

z_stat, pval = proportions_ztest(successes, nobs=nobs)
(lower_con, lower_treat), (upper_con, upper_treat) = proportion_confint(successes, nobs=nobs, alpha=0.05)

print(f'Z test stat: {z_stat:.2f}')
print(f'P-value: {pval:.3f}')
print(f'Confidence Interval of 95% for control group: [{lower_con:.3f}, {upper_con:.3f}]')
print(f'Confidence Interval of 95% for treatment group: [{lower_treat:.3f}, {upper_treat:.3f}]')

## 7. Summarise results and explain your answers

The change to the homepage slightly decreased the click through to the log in page. 

The p-value is well over the Alpha value of 0.05, meaning the null hypothesis cannot be rejected. 